In [2]:
import cv2
import numpy as np
from numpy import linalg as LA

In [3]:
def l1_distance(x, y):
    return abs(x-y)

def l2_distance(x, y):
    return (x - y)**2

In [4]:
left_img_path = 'tsukuba/left.png'
right_img_path = 'tsukuba/right.png'
disparity_range = 16
max_value = 255
scale = 16

# PIXEL-WISE MATCHING

In [5]:
# Using pixel wise matching 
def pixel_wise_matching(left_img_path, right_img_path, disparity_range, scale):
    # Grayscale
    left = cv2.imread(left_img_path, 0)
    # left = cv2.cvtColor(left, cv2.COLOR_BGR2GRAY)
    right = cv2.imread(right_img_path, 0)

    # Type Conversion
    left = left.astype(np.float32)
    right = right.astype(np.float32)

    height, width = left.shape

    # Create disparity map
    depth = np.zeros(shape=(height, width), dtype= np.uint8)
    
    for h in range(height):
        for w in range(width):
            costs = []
            for d in range(disparity_range):
                if w - d < 0:
                    cost = max_value
                else:
                    cost = abs(
                            left[h, w] - right[h, w - d]
                            )
                costs.append(cost)
            min_cost = min(costs)
            min_cost_idx = costs.index(min_cost) # disparity

            depth[h, w] = min_cost_idx * scale

    return depth

In [14]:
depth = pixel_wise_matching(left_img_path, right_img_path, disparity_range, scale)  
depth = cv2.applyColorMap(depth, cv2.COLORMAP_JET)
cv2.imwrite('pixel_wise.png', depth)
cv2.imshow('depth',depth)
cv2.waitKey(0)
cv2.destroyAllWindows()

# WINDOW-BASED MATCHING

In [7]:
def window_based_matching(left_img_path, right_img_path, disparity_range=16, scale=3, max_value=255*9, kernel_size=3):
    
    left = cv2.imread(left_img_path, 0)
    right = cv2.imread(right_img_path, 0)

    # Type Conversion
    left = left.astype(np.float32)
    right = right.astype(np.float32)

    height, width = left.shape

    # Create disparity map
    depth = np.zeros(shape=(height, width), dtype= np.uint8)
    
    kernel_half = int((kernel_size - 1) / 2)
    scale = scale
    max_value = max_value

    for y in range(kernel_half, height - kernel_half + 1):
        for x in range(kernel_half, width - kernel_half + 1):

            disparity = 0
            cost_min = max_value

            for j in range(disparity_range):
                total = 0
                value = 0

                for v in range(-kernel_half, kernel_half):
                    for u in range(-kernel_half, kernel_half):
                        value = max_value
                        if (x + u - j) >= 0:
                            value = l1_distance(
                                int(left[y + v, x + u]), int(right[y + v, (x + u) - j])
                            )
                            total += value
                
                if total < cost_min:
                    cost_min = total
                    disparity = j
            
            depth[y, x] = disparity * scale
    
    return depth

    

In [13]:
depth_1 = window_based_matching(left_img_path, right_img_path, scale = 16)  
depth_1 = cv2.applyColorMap(depth_1, cv2.COLORMAP_JET)
cv2.imwrite('window_based.png', depth_1)
cv2.imshow('depth',depth_1)
cv2.waitKey(0)
cv2.destroyAllWindows()

# WINDOW-BASED MATCHING with Cosine Similarity

In [9]:
def cosine_similarity(x, y):
    numerator = np.dot(x,y)
    denominator = LA.norm(x) * LA.norm(y)
    return numerator / denominator

In [10]:
def window_based_cos_similarity(left_img_path, right_img_path, disparity_range=16, scale=3,  kernel_size=3):
    left = cv2.imread(left_img_path, 0)
    right = cv2.imread(right_img_path, 0)

    # Type Conversion
    left = left.astype(np.float32)
    right = right.astype(np.float32)

    height, width = left.shape

    # Create disparity map
    depth = np.zeros(shape=(height, width), dtype= np.uint8)
    
    kernel_half = int((kernel_size - 1) / 2)
    scale = scale
    
    for y in range(kernel_half, height - kernel_half + 1):
        for x in range(kernel_half, width - kernel_half + 1):

            disparity = 0
            cost_optimal = -1

            for j in range(disparity_range):
                d = x - j
                cost = 0
                if (d - kernel_half) > 0:
                    wp = left[(y-kernel_half):(y+kernel_half)+1, (x-kernel_half):(x+kernel_half)+1]
                    wqd = right[(y-kernel_half):(y+kernel_half)+1, (x-kernel_half):(x+kernel_half)+1]

                    wp_flattened = wp.flatten()
                    wqd_flattened = wqd.flatten()

                    cost = cosine_similarity(wp_flattened, wqd_flattened)

                if cost > cost_optimal:
                    cost_optimal = cost
                    disparity = j
                
                depth[y, x] = disparity * scale
                
    return depth


In [11]:
depth_2 = window_based_cos_similarity(left_img_path, right_img_path, scale = 16)
depth_2 = cv2.applyColorMap(depth_2, cv2.COLORMAP_JET)
cv2.imshow('depth',depth_2)
cv2.waitKey(0)
cv2.destroyAllWindows()